In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [4]:
# !wget https://f000.backblazeb2.com/file/malaya-model/v37/translation/ms-en/base-translation.pb
# !wget https://f000.backblazeb2.com/file/malaya-model/v37/translation/ms-en/small-translation.pb
# !wget https://f000.backblazeb2.com/file/malaya-model/v37/translation/ms-en/large-translation.pb

In [5]:
import tensorflow as tf
from tensorflow.tools.graph_transforms import TransformGraph
from glob import glob
tf.compat.v1.set_random_seed(0)

In [8]:
pbs = glob('*.pb')
pbs

['base-translation.pb', 'large-translation.pb', 'small-translation.pb']

In [9]:
import tensorflow_text
import tf_sentencepiece

In [10]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_constants(ignore_errors=true)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

for pb in pbs:
    input_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v1.gfile.FastGFile(pb, 'rb') as f:
        input_graph_def.ParseFromString(f.read())
        
    print(pb)
    
    transformed_graph_def = TransformGraph(input_graph_def, 
                                           ['Placeholder'],
                                           ['greedy', 'beam'], transforms)
    
    with tf.compat.v1.gfile.GFile(f'{pb}.quantized', 'wb') as f:
        f.write(transformed_graph_def.SerializeToString())

Instructions for updating:
Use tf.compat.v1.gfile.GFile.
base-translation.pb
large-translation.pb
small-translation.pb


In [11]:
quantized = glob('*.pb.quantized')
quantized

['base-translation.pb.quantized',
 'small-translation.pb.quantized',
 'large-translation.pb.quantized']

In [15]:
!rm *.pb*

rm: cannot remove '*.pb*': No such file or directory


In [ ]:
# converter = @@#lite.TFLiteConverter.from_frozen_graph(
#     graph_def_file='test.pb',
#     input_arrays=['Placeholder', 'Placeholder_1'],
#     input_shapes={'Placeholder' : [None, 512], 'Placeholder_1': [None, 512]},
#     output_arrays=['logits'],
# )
# # converter.allow_custom_ops=True

In [ ]:
# converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS, tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_types = [tf.compat.v1.float16]
# converter.optimizations = [tf.compat.v1.lite.Optimize.DEFAULT]
# converter.experimental_new_converter = True
# tflite_model = converter.convert()

In [ ]:
# converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS, 
#                                        tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_types = [tf.compat.v1.float16]
# converter.optimizations = [tf.compat.v1.lite.Optimize.DEFAULT]
# tflite_model = converter.convert()

# with open('tiny-bert-sentiment-float16.tflite', 'wb') as f:
#     f.write(tflite_model)

In [ ]:
# converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS, 
#                                        tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]
# converter.optimizations = [tf.compat.v1.lite.Optimize.OPTIMIZE_FOR_SIZE]
# tflite_model = converter.convert()

# with open('tiny-bert-sentiment-hybrid.tflite', 'wb') as f:
#     f.write(tflite_model)

In [ ]:
# interpreter = tf.compat.v1.lite.Interpreter(model_path='tiny-bert-sentiment-hybrid.tflite')
# interpreter.allocate_tensors()